In [45]:
#how to tabulate the occurence of each digits 0-9 from set of numbers 5/75 and 1/15
parseStr <- ''
for (i in 1:75)
  parseStr <- paste(parseStr,i,sep="")
for (i in 1:15)
  parseStr <- paste(parseStr,i,sep="")
#print(parseStr)

cols <- 10
mat <- matrix(NA, nrow=1, ncol=cols)
for (n in 1:cols)  
    mat[,n] <- c(n-1)    
mat2 <- matrix(NA, nrow=1, ncol=cols)
for (n in 1:cols)
    mat2[,n] <- c(lengths(regmatches(parseStr, gregexpr(n-1, parseStr))))

cmatrix <-rbind(mat,mat2)
#print(cmatrix)
for (n in 1:cols)     
    print(paste(cmatrix[1,n],cmatrix[2,n],sep=",")    )

[1] "0,8"
[1] "1,26"
[1] "2,20"
[1] "3,20"
[1] "4,20"
[1] "5,20"
[1] "6,18"
[1] "7,14"
[1] "8,8"
[1] "9,8"


In [76]:
##### library(dplyr)
# load the MASS package and suppress warning about masked functions
library(MASS, warn.conflicts = FALSE)            

# Copied a function for mode since R has no build in function for mode???
# thanks to http://stackoverflow.com/users/169947/ken-williams who answered this in stackoverflow

getmode <- function(v) {
   uniqv <- unique(v)
   uniqv[which.max(tabulate(match(v, uniqv)))]
}
getmode2 <- function(v) {
   uniqv <- unique(v)
   uniqv <- uniqv[uniqv != getmode(v)]
   uniqv[which.max(tabulate(match(v, uniqv)))]
}
#download the latest file at http://www.calottery.com/sitecore/content/Miscellaneous/download-numbers/?GameName=mega-millions&Order=No
#save to local C: drive
myData <- read.csv("C:/megamillions.csv")

#convert date string to date
myData$Draw.Date <- as.Date(myData$Draw.Date,  "%a. %b %d, %Y")  

#add column on day of the draw (2-Tues, 5-Frid)
myData$Draw.Day <- as.integer(strftime(myData$Draw.Date,'%u'))   

#megamillions changed the number set to 1-75 in Oct 22, 2013 and decreased the odds of winning
#prior Oct 18, 2013: 5/56 + Mega: 1/46   after Oct 18, 2013: 5/75 + Mega: 1/15 
df <- subset(myData, myData$Draw.Date > as.Date('2013-10-19'))   

#parse all winnings numbers into a string
df$CombinedNum <- paste(df$Number_1,df$Number_2,df$Number_3,df$Number_4,df$Number_5,df$Mega,sep="")
#get the summ of all winning numbers per draw
df$CombinedSum <- (df$Number_1+df$Number_2+df$Number_3+df$Number_4+df$Number_5+df$Mega)
str(df)

#get the length of parsed numbers
df$CombinedSize <- nchar(df$CombinedNum)

#create new columns to count how many times a number (0-9) occurs
df$Count_1 <- lengths(regmatches(df$CombinedNum, gregexpr("1", df$CombinedNum)))
df$Count_2 <- lengths(regmatches(df$CombinedNum, gregexpr("2", df$CombinedNum)))
df$Count_3 <- lengths(regmatches(df$CombinedNum, gregexpr("3", df$CombinedNum)))
df$Count_4 <- lengths(regmatches(df$CombinedNum, gregexpr("4", df$CombinedNum)))
df$Count_5 <- lengths(regmatches(df$CombinedNum, gregexpr("5", df$CombinedNum)))
df$Count_6 <- lengths(regmatches(df$CombinedNum, gregexpr("6", df$CombinedNum)))
df$Count_7 <- lengths(regmatches(df$CombinedNum, gregexpr("7", df$CombinedNum)))
df$Count_8 <- lengths(regmatches(df$CombinedNum, gregexpr("8", df$CombinedNum)))
df$Count_9 <- lengths(regmatches(df$CombinedNum, gregexpr("9", df$CombinedNum)))
df$Count_0 <- lengths(regmatches(df$CombinedNum, gregexpr("0", df$CombinedNum)))

#created a new data frame with 13 columns from the right
newDf <- df[, c(tail(seq_len(ncol(df)), 13))]
head(newDf)

#apply the same technique of parsing the counts
newDf$combinedCounts <- paste(df$Count_1,df$Count_2,df$Count_3,df$Count_4,df$Count_5,df$Count_6,df$Count_7,df$Count_8,df$Count_9,df$Count_0,sep="")

#same with counting earlier
newDf$combCount_0 <- lengths(regmatches(newDf$combinedCounts, gregexpr("0", newDf$combinedCounts)))
newDf$combCount_1 <- lengths(regmatches(newDf$combinedCounts, gregexpr("1", newDf$combinedCounts)))
newDf$combCount_2 <- lengths(regmatches(newDf$combinedCounts, gregexpr("2", newDf$combinedCounts)))
newDf$combCount_3 <- lengths(regmatches(newDf$combinedCounts, gregexpr("3", newDf$combinedCounts)))
newDf$combCount_4 <- lengths(regmatches(newDf$combinedCounts, gregexpr("4", newDf$combinedCounts)))

#######end of creating columns #########
#      start of data analyis  
########################################
colSum <- newDf$CombinedSum
breaks <- seq(10, 310, by=40)
sumOfNumbers.freq.dist <- cut(colSum, breaks, right=FALSE)
table(sumOfNumbers.freq.dist)
sprintf("This is the suggested sum of numbers: %s", '[170,210)')
sprintf("This is the most common CombinedSum: %s", getmode(newDf$CombinedSum))

#display the chance of each digit in set of the winning numbers
cMatrixLbl <- matrix(c('Number','Chance'),nrow=2,ncol=1)

#tabulate how many ways can 0-9 appears in the winning numbers
cMatrix <- matrix(c(0,8,1,26,2,20,3,20,4,20,5,20,6,18,7,14,8,8,9,8),nrow=2,ncol=10)
cMatrix[2,] <- paste(round(cMatrix[2,]/sum(cMatrix[2,])*100,digits=2),'%')
cMatrixFinal <- cbind(cMatrix,cMatrixLbl)
cMatrixFinal

#apply the table function
table(df$CombinedSize)   
sprintf("This is the suggested size of numbers: %s", getmode(df$CombinedSize))

finalDf <- newDf[, c(tail(seq_len(ncol(newDf)), 6))]
finalDf$sumSq <-  (newDf$combCount_0^2+newDf$combCount_1^2+newDf$combCount_2^2+newDf$combCount_3^2+newDf$combCount_4^2)

sprintf("This is the most common sum of patterns: %s", getmode(finalDf$sumSq))

sprintf("This is the most common combinedCounts: %s", getmode(newDf$combinedCounts))
cntCombCnts <- nrow(filter(newDf, combinedCounts == getmode(newDf$combinedCounts)))
sprintf("how many times?: %s", cntCombCnts)

newDf$cntCombNum <- paste(newDf$combCount_0,newDf$combCount_1,newDf$combCount_2,newDf$combCount_3,newDf$combCount_4,sep="")
sprintf("This is the most common patterns of combined counts: %s", getmode(newDf$cntCombNum))
cntCombNum <- nrow(filter(newDf, cntCombNum == getmode(newDf$cntCombNum)))
sprintf("how many times?: %s", cntCombNum)

subfinaldf <- subset(newDf, newDf$cntCombNum == getmode(newDf$cntCombNum))
subpatterns <- subfinaldf[, c(tail(seq_len(ncol(subfinaldf)), 7))]
sprintf("Most pattern is below, use these") 
head(subpatterns$combinedCounts)

########## summary ###########
print("Facts:")
print("Number 1 is 16% chance to appear; 2,3,4,5 and 6 is 12%; 9 is 9% and 7,8 and 0 is 5%")
print("Average sum is around 198: min 16 amd max 380")
print("")
print("SUMMARY of Insights:")
print("Sum is between 170 and 210: 35%")
print("The pattern is 3 missing digits from 7-9,0, 3 digits occuring once, 2 twice, 1 once: 33%")
print("Only one one-digit number including mega number: 46%")
print("for ex: below set of numbers has a sum=164; pattern=2113101010 and one one-digit number")
print("11 17 30 66 58 Mega:3")
print("Counter check at http://www.calottery.com/play/draw-games/mega-millions/winning-numbers")
print("mostly 3 numbers are correct")

print("")
print("NEXT STEP: Generate set of numbers based on bday, favorite number, anniversary with below properties")
print("Sum is 172, has one one-digit and pattern 2113101010")



'data.frame':	351 obs. of  11 variables:
 $ Draw..     : int  1220 1219 1218 1217 1216 1215 1214 1213 1212 1211 ...
 $ Draw.Date  : Date, format: "2017-02-28" "2017-02-24" ...
 $ Number_1   : int  33 68 32 67 7 32 28 3 3 61 ...
 $ Number_2   : int  69 29 75 4 11 39 37 30 27 53 ...
 $ Number_3   : int  58 12 30 75 68 51 23 6 14 54 ...
 $ Number_4   : int  45 42 21 56 60 62 71 64 72 17 ...
 $ Number_5   : int  20 33 9 58 33 75 56 29 62 37 ...
 $ Mega       : int  4 14 9 8 15 14 12 3 4 8 ...
 $ Draw.Day   : int  2 5 2 5 2 5 2 5 2 5 ...
 $ CombinedNum: chr  "33695845204" "682912423314" "3275302199" "6747556588" ...
 $ CombinedSum: int  229 198 176 268 194 273 227 135 182 230 ...


CombinedNum,CombinedSum,CombinedSize,Count_1,Count_2,Count_3,Count_4,Count_5,Count_6,Count_7,Count_8,Count_9,Count_0
33695845204,229,11,0,1,2,2,2,1,0,1,1,1
682912423314,198,12,2,3,2,2,0,1,0,1,1,0
3275302199,176,10,1,2,2,0,1,0,1,0,2,1
6747556588,268,10,0,0,0,1,3,2,2,2,0,0
71168603315,194,11,3,0,2,0,1,2,1,1,0,1
323951627514,273,12,2,2,2,1,2,1,1,0,1,0


sumOfNumbers.freq.dist
  [10,50)   [50,90)  [90,130) [130,170) [170,210) [210,250) [250,290) 
        0         2        15        91       122        82        32 

[1] "This is the suggested sum of numbers: [170,210)"

[1] "This is the most common CombinedSum: 172"

...

[1] "This is the most common patterns of combined counts: 34210"

[1] "how many times?: 24"

[1] "Most pattern is below, use these"

[1] "3020121101" "2101023110" "0110312210" "1312020101" "1202311100"
[6] "1202113100"

[1] "Facts:"
[1] "Number 1 is 16% chance to appear; 2,3,4,5 and 6 is 12%; 9 is 9% and 7,8 and 0 is 5%"
[1] "Average sum is around 198: min 16 amd max 380"
[1] ""
[1] "SUMMARY of Insights:"
[1] "Sum is between 170 and 210: 35%"
[1] "The pattern is 3 missing digits from 7-9,0, 3 digits occuring once, 2 twice, 1 once: 33%"
[1] "Only one one-digit number including mega number: 46%"
[1] "for ex: below set of numbers has a sum=164; pattern=2113101010 and one one-digit number"
[1] "11 17 30 66 58 Mega:3"
[1] "Counter check at http://www.calottery.com/play/draw-games/mega-millions/winning-numbers"
[1] "mostly 3 numbers are correct"
[1] ""
[1] "NEXT STEP: Generate set of numbers based on bday, favorite number, anniversary with below properties"
[1] "Sum is 172, has one one-digit and pattern 2113101010"
